In [1]:
import numpy as np
import os
import scipy.io as scio

In [2]:
def process_data(data):
    # average_data = np.mean(data[:, :, ::2], axis=2)
    data1 = data[:, :, ::2]  # 偶
    data2 = data[:, :, 1::2] # 奇

    return (data1 + data2)

HeatCubes

In [6]:
matfile = ''
root = '/home/ljm/workspace/zx/HADAR_database/'
out_root = '/home/ljm/workspace/zx/HADAR_database1/'
for i in list(range(1,11)):
    filename = os.listdir(os.path.join(root, f'Scene{i}', 'HeatCubes'))
    out_filename = os.path.join(out_root, f'Scene{i}', 'HeatCubes')
    if not os.path.exists(out_filename):
        os.makedirs(out_filename)
    # print(filename)
    # print(len(filename))
    for file in filename:
        if file[-3:] == 'mat':
            matfile = os.path.join(root, f'Scene{i}', 'HeatCubes') + '/' + file
            data = np.array(scio.loadmat(matfile)["S"])
            data_ = process_data(data)
            scio.savemat(out_filename + '/' + file, {'S': data_})
            # print(filename)
            print(out_filename + '/' + file + ' is saved')
            

/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/L_0001_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/R_0002_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/R_0004_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/L_0003_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/R_0003_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/R_0001_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/L_0005_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/R_0005_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/L_0002_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/L_0004_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene2/HeatCubes/L_0001_heatcube.matis saved
/home/ljm/workspace/zx/HADAR_database1/Scene2/HeatCubes/R_0002_heatcube.mati

### .mat to .npy
读取.mat

In [5]:
import time
file = '/home/ljm/workspace/zx/HADAR_database/Scene1/HeatCubes/L_0001_heatcube.mat'
mat_start_time = time.time()
mat = scio.loadmat(file)["S"]
mat_end_time = time.time()
print(mat_end_time - mat_start_time)


1.5963153839111328


保存.npy

In [9]:
save_dir = '/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
np.save(os.path.join(save_dir, 'L_0001_heatcube.npy'), mat)

In [11]:
npy_start_time = time.time()
npy = np.load(os.path.join(save_dir, 'L_0001_heatcube.npy'))
npy_end_time = time.time()
print(npy_end_time - npy_start_time)

0.2799663543701172


In [12]:
npy == mat

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr

## 读取mat文件名列表

In [2]:
matfile = []
root = '~/workspace/zx/HADAR_database/'
for i in list(range(1,11)):
    filename = os.listdir(os.path.join(root, f'Scene{i}', 'HeatCubes'))
# print(filename)
# print(len(filename))
    for file in filename:
        if file[-3:] == 'mat':
            matfile.append(os.path.join(root, f'Scene{i}', 'HeatCubes') + '/' + file)
print(matfile)
print(len(matfile))

['/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/R_0001_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/L_0001_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/R_0004_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/R_0002_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/R_0003_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/L_0005_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/L_0003_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/L_0002_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/L_0004_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene1/HeatCubes/R_0005_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene2/HeatCubes/R_0001_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene2/HeatCubes/L_0001_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Scene2/HeatCubes/R_0004_heatcube.mat', '/mnt/Disk/zx/HADAR/HADAR_database/Sc

### 读取数据

In [3]:
data = []
for mat in matfile:
    sub_data = scio.loadmat(mat)["S"]
    data.append(sub_data)
data = np.array(data)
print(data.shape)


(100, 1080, 1920, 54)
